In [ ]:
#importing modules
import cv2
import numpy as np

In [ ]:
# initialising values
whT = 320
confThreshold = 0.5

In [ ]:
#loadimg obj names
classesFile = 'obj.names'
className = []
with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

In [ ]:
#importing yolo config and weighs
modelConfiguration = 'yolov4-tiny-custom.cfg'
modelWeights = 'yolov4-tiny-custom_best.weights'

In [ ]:
#setting backend and target
net = cv2.dnn.readNet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [ ]:
#function for object detection 
def findObjects(output, img):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []

    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w, h = int(det[2]*wT), int(det[3]*hT)
                x, y = int((det[0]*wT)-w/2), int((det[1]*hT)-h/2)
                bbox.append([x, y, w, h])
                classIds.append(classId)
                confs.append(float(confidence))

    indices = cv2.dnn.NMSBoxes(bbox, confs, confThreshold, nms_threshold=0.3)
    print(indices)
    for i in indices:
        i = i[0]
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 255), 2)
        cv2.putText(img, f'{classNames[classIds[i]].upper()} {int(confs[i]*100)}%',
                    (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)

In [ ]:
#opencv code for loading video

cap = cv2.VideoCapture('V.mp4')
while True :
#     img = cv2.imread('v.jpg')
    success, img = cap.read()
    img = cv2.resize(img, (600,600))
    blob = cv2.dnn.blobFromImage(img, 1/255, (whT, whT), [0, 0, 0], 1, crop= False)
    net.setInput(blob)

    layerNames = net.getLayerNames()
    outNames = [layerNames[i[0]-1] for i in net.getUnconnectedOutLayers()]
    print(outNames)
    
    outputs = net.forward(outNames)
    findObjects(outputs, img)

    cv2.imshow('image', img)
    if cv2.waitKey(1) == 27:
        break
cap.release()
cv2.destroyAllWindows()